In [1]:
import multiprocessing as mp
import time
import os
import gc

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import log_loss, auc, roc_curve, f1_score, average_precision_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold
import catboost as cat

from reduce_memory import reduce_numeric_mem_usage, reduce_object_mem_usage

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [17]:
sub_sample = pd.read_csv('./data/submission_popular.csv')

In [2]:
test_pred = pd.read_csv('./data/sub.csv')

In [27]:
test = pd.read_csv('./data/test.csv')

In [3]:
test_pred.head()

,user_id,session_id,timestamp,step,item_recommendations,recommendation
0,000324D9BBUC,89643988fdbfb,1541593942,10,924795 106315 1033140 119494 101758 903037 105...,923407 1050792 106315 97171 924795 353141 1033...
1,0004Q49X39PY,9de47d9a66494,1541641157,1,3505150 3812004 2227896 2292254 3184842 222702...,2213014 3184842 3812004 3505150 10213134 22922...
2,0004Q49X39PY,beea5c27030cb,1541561202,1,4476010 3505150 3812004 2227896 2292254 222702...,3812004 3505150 2292254 4476010 7101352 222789...
3,00071784XQ6B,9617600e1ba7c,1541630328,2,22854 3067559 22721 22713 16121 22772 22727 22...,22854 22721 3067559 16121 22764 22819 22713 22...
4,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,9857656 5849628 655716 1352530 502066 1405084 ...,507861 2176280 1669587 9857656 4272108 502066 ...


In [72]:
test_pred.drop('item_recommendations', axis=1, inplace=True)

In [75]:
test_pred.rename(columns={'recommendation': 'item_recommendations'}, inplace=True)

In [76]:
test_pred.to_csv('first_sub.csv', index=False)

In [4]:
test_pred.iloc[0]['item_recommendations']

'924795 106315 1033140 119494 101758 903037 1050792 97171 353141 2094950 923407 106307 1000915 97170 147907 1838901 1729121 2182130 904851 927351 4234718 2358690 2431104 1241375 152107'

In [5]:
test_pred.iloc[0]['recommendation']

'923407 1050792 106315 97171 924795 353141 1033140 2094950 106307 903037 1000915 1838901 119494 1241375 147907 101758 97170 1729121 2182130 4234718 927351 2358690 904851 152107 2431104'

In [10]:
sub_sample.to_csv('submission.csv', index=False)

---

In [69]:
test.reference.nunique()

184878

In [ ]:
def output_impressions(df):
    pred_list = df['pred'].values
    sorted_ind = np.argsort(pred_list)[::-1]
    ranked_items = list(df['item_id'].iloc[sorted_ind].values)
    ranked_items_str = [str(i) for i in ranked_items]
    return ' '.join(ranked_items_str)
xtest['pred'] = np.random.randint(0,2, len(xtest))

In [ ]:
test_imps_pred = xtest.groupby(level=0).apply(output_impressions)
#     test_imps_pred = xtest.groupby(level=0).apply(output_impressions)
test_imps_pred = test_imps_pred.reset_index(name='recommendation')

In [ ]:
test_imps_pred.head()

In [ ]:
sub = pd.read_csv('./data/submission_popular.csv')


In [ ]:
sub.head()

In [ ]:
subs = pd.merge(sub, test_imps_pred, how='left', on='session_id')


In [ ]:
subs.head()

In [ ]:
set(test_imps_pred.session_id).intersection(subs.user_id)

In [ ]:
set(xtest.session_id).intersection(subs.session_id)

In [ ]:
xtest.index

In [ ]:
test = pd.read_csv('./data/test.csv')

In [ ]:
test.head()

In [ ]:
test[test.session_id=='1d688ec168932']

In [ ]:
len(sub)

In [ ]:
sub.session_id.isin(test.session_id).sum()

In [ ]:
len(subs)

In [ ]:
xtest[xtest.session_id=='89643988fdbfb']

In [ ]:
sub_sample[sub_sample.session_id=='89643988fdbfb']

In [ ]:
xval = pd.read_hdf('./data/train.h5', key='xval')

In [ ]:
xval.head()

In [ ]:
%%time
train = pd.read_csv('./data/train.csv')

In [ ]:
train.head()

In [ ]:
xval.head()

In [ ]:
xval.session_id.isin(train.session_id).sum()

In [ ]:
xval.shape

In [11]:
tc = pd.read_csv('./test_click.csv')

In [12]:
tc.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,ts
0,P2L70S1Y60IF,0000059a39020,1541719950,1,clickout item,NaN,JP,"Sapporo, Japan",mobile,NaN,2251200|924581|4775012|10090928|2282660|895299...,54|42|64|51|62|42|61|54|53|56|54|53|57|59|44|4...,2018-11-08 23:32:30
1,SKDEGHT3I151,0000b9394297b,1541606075,1,search for poi,Sassi di Matera,AR,"Matera, Italy",mobile,NaN,NaN,NaN,2018-11-07 15:54:35
2,SKDEGHT3I151,0000b9394297b,1541606244,2,clickout item,NaN,AR,"Matera, Italy",mobile,NaN,1044228|5670638|5875672|6033244|7917004|507857...,242|193|288|180|196|129|155|232|245|191|188|17...,2018-11-07 15:57:24
3,VEGQ0VOVB802,0000e0500c2e7,1541585429,1,clickout item,3844380,IN,"Saputara, India",mobile,NaN,6688304|3844380|1542171|3793522|2656440|260528...,46|25|48|71|132|44|71|8|32|57|54|66|57|33|57|7...,2018-11-07 10:10:29
4,VEGQ0VOVB802,0000e0500c2e7,1541585739,2,clickout item,9387530,IN,"Saputara, India",mobile,NaN,6688304|3844380|1542171|3793522|2656440|260528...,46|25|48|71|132|44|71|8|32|57|54|66|57|33|57|7...,2018-11-07 10:15:39


In [13]:
tc.groupby('session_id').tail(1)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,ts
0,P2L70S1Y60IF,0000059a39020,1541719950,1,clickout item,NaN,JP,"Sapporo, Japan",mobile,NaN,2251200|924581|4775012|10090928|2282660|895299...,54|42|64|51|62|42|61|54|53|56|54|53|57|59|44|4...,2018-11-08 23:32:30
2,SKDEGHT3I151,0000b9394297b,1541606244,2,clickout item,NaN,AR,"Matera, Italy",mobile,NaN,1044228|5670638|5875672|6033244|7917004|507857...,242|193|288|180|196|129|155|232|245|191|188|17...,2018-11-07 15:57:24
4,VEGQ0VOVB802,0000e0500c2e7,1541585739,2,clickout item,9387530,IN,"Saputara, India",mobile,NaN,6688304|3844380|1542171|3793522|2656440|260528...,46|25|48|71|132|44|71|8|32|57|54|66|57|33|57|7...,2018-11-07 10:15:39
5,V3SQM6X57BNX,0001650138d30,1541614783,1,clickout item,NaN,CA,"Banff, Canada",mobile,NaN,44394|44103|44247|44155|43972|44292|44404|4449...,259|220|99|210|283|252|152|192|179|213|199|175...,2018-11-07 18:19:43
9,13UA01X5MOHZ,000177b850519,1541574580,4,clickout item,NaN,TR,"Rome, Italy",desktop,NaN,17011|16122|17403|17040|17266|17295|17535|1757...,875|726|216|396|263|174|182|264|327|265|364|27...,2018-11-07 07:09:40
20,K0LALBSISIAC,00017b3b2c136,1541565393,11,clickout item,NaN,CO,"Piedecuesta, Colombia",mobile,NaN,5723818|4341718|6298194|6698142|7952264|285957...,46|89|21|14|59|74|37|33|30|46|17|77|12|37|44|3...,2018-11-07 04:36:33
26,51BGO9LGJDHT,0001d7853e07d,1541704166,6,clickout item,NaN,IE,"Killarney, Ireland",tablet,NaN,19822|854066|888605|19821|19820|906385|485171|...,115|59|89|95|134|220|110|89|119|190|143|195|10...,2018-11-08 19:09:26
27,0ROCDBG2IZEE,000227a856f46,1541644962,1,clickout item,NaN,IN,"Dapoli, India",mobile,NaN,2667568|3825952|2661488|6940638|9793448|977722...,92|43|61|250|48|34|10|27|36|274|52|463|110,2018-11-08 02:42:42
28,KER7PAE786J1,000243bb22a50,1541612425,1,clickout item,NaN,DE,"Paris, France",mobile,NaN,49536|149808|2418418|1122314|149795|5502530|62...,82|40|44|48|49|49|52|52|55|55|56|56|56|58|58|5...,2018-11-07 17:40:25
61,2U7433585MSR,000246890bfc6,1541717372,33,clickout item,NaN,NZ,"Wellington, New Zealand",tablet,Sort by Price,1257157|7845384|2743534|1276700|1276698|790286...,32|40|47|49|50|61|63|64|71|74|76|83|85|86|87|8...,2018-11-08 22:49:32
